In [ ]:
#importi

In [ ]:
model = load_model("./modeli/model_1.keras")

In [ ]:
# evaluate the network
print("[INFO] evaluating the network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1),
                            target_names=labelNames))

# save the network to disk 
print("serializing the network '{}'...".format(args["model"]))
model.save(args["model"])

In [ ]:
args = {
    "model": "/kaggle/working/trafficsignnet.model",
    "images": "/kaggle/input/gtsrb-german-traffic-sign/Test",
    "examples" : "/kaggle/working/examples"
}

# load the traffic sign recognizer model
print("[INFO] loading model...")
model = load_model(args["model"])

# load the label names
labelNames = open("/kaggle/input/signnames/signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

# grab the paths to the input images, shuffle them, and grab a sample
print("[INFO] predicting...")
imagePaths = list(paths.list_images(args["images"]))
random.shuffle(imagePaths)
imagePaths = imagePaths[:25]

In [ ]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # load the image, resize it to 32x32 pixels, and then apply
    # Contrast Limited Adaptive Histogram Equalization (CLAHE),
    # just like we did during training
    image = io.imread(imagePath)
    image = transform.resize(image, (32, 32))
    image = exposure.equalize_adapthist(image, clip_limit=0.1)

    # preprocess the image by scaling it to the range [0, 1]
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)

    # make predictions using the traffic sign recognizer CNN
    preds = model.predict(image)
    j = preds.argmax(axis=1)[0]
    label = labelNames[j]

    # load the image using OpenCV, resize it, and draw the label
    # on it
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=128)
    cv2.putText(image, label, (5, 15), cv2.FONT_HERSHEY_SIMPLEX,
        0.45, (0, 0, 255), 2)

    # save the image to disk
    p = os.path.sep.join([args["examples"], "{}.png".format(i)])
    cv2.imwrite(p, image)

In [ ]:
# Reload the model
model = load_model('./road_sign.h5')

# Get the predictions for the test data set
predicted_classes = np.argmax(cnn_model.predict(X_test_grscale_norm), axis=-1)

# Retrieve the indices that we will plot
y_true = y_test

# Plot some of the predictions on the test data set
for i in range(15):
  plt.subplot(5,3,i+1)
  plt.imshow(X_test_grscale_norm[i].squeeze(), 
    cmap='gray', interpolation='none')
  plt.title("Predict {}, Actual {}".format(predicted_classes[i], 
    y_true[i]), fontsize=10)
plt.tight_layout()
plt.savefig('road_sign_output.png')
plt.show()

In [ ]:
def sign_predict(image):
    model = load_model('./keras_model/')    
    image = np.array(image, dtype=np.float32)    
    image = image/255    
    image = np.reshape(image, (1, 32, 32))    
    x = image.astype(np.float32)    
    prediction = model.predict(x)    
    prediction_max = np.argmax(prediction)    
    prediction_label = labels_dict[prediction_max]    
    confidence = np.max(prediction)    
    return prediction_label, confidence

In [ ]:
import sys
import glob
import matplotlib.image as mpimg

# Download at least 5 pictures of German traffic signs from the web
# Street view of germany in google maps
X_test_web = []
filenames = []
files = glob.glob('assets/data/images/web_gts/*')
for file in files:
    if sys.platform == 'win32':
        filename = file.split('\\')[-1]
    elif sys.platform == 'linux':
        filename = file.split('/')[-1]
    filenames.append(filename)
    image = mpimg.imread(file)
    X_test_web.append(image)

# load class_ID:sign_name as key:value pair into dictionary
web_sign_dict = {}
with open("assets/data/images/web_signnames.csv") as signnames_file:
    reader = csv.reader(signnames_file)
    # Skip the first row of the csv file
    next(reader)    
    for rows in reader:
        web_sign_dict[int(rows[1])] = rows[2]

web_sign_label = list(web_sign_dict.keys())
web_sign_name = list(web_sign_dict.values())
    
n_web_images = len(X_test_web)

# increasing col, increases size of images in subplot. col = 60
fig = plt.figure(figsize=(30,20))
fig.subplots_adjust(hspace = 0.05, wspace = 0.2)
nrows = math.ceil(n_web_images/3) # 2 rows
ncols = math.ceil(n_web_images/2) # 3 columns
# Plot the web images
for i in range(n_web_images):
    sub = fig.add_subplot(nrows, ncols, i + 1)
    image = X_test_web[i]
    title = "{}: {}".format(web_sign_label[i], web_sign_name[i])
    sub.set_title(title, size=35)
    sub.imshow(image, interpolation = 'nearest')
    
fig.savefig('assets/data/images/output/web_test_images.jpg', bbox_inches='tight')

In [ ]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
pred = model.predict_classes(X_test)
#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))
model.save(‘traffic_classifier.h5’)

In [ ]:
from sklearn.metrics import accuracy_score
test = pd.read_csv("/kaggle/input/gtsrb-german-traffic-sign/Test.csv")
test_labels = test['ClassId'].values.tolist()

test_img_path = "/kaggle/input/gtsrb-german-traffic-sign"
test_imgs = test['Path'].values
test_data = []

for img in test_imgs:
    im = Image.open(test_img_path + '/' + img)
    im = im.resize((30,30))
    im = np.array(im)
    test_data.append(im)
test_data = np.array(test_data)
predictions = model.predict(test_data)
classes_x = np.argmax(predictions, axis = 1).tolist()
classes_x = np.array([classes_x]).tolist()[0]

In [ ]:
print("accuracy: ", accuracy_score(test_labels, classes_x))

In [ ]:
number_of_image_test = 7 #20

test_image_name = test_imgs[number_of_image_test]
image_label = test_labels[number_of_image_test]
plt.figure(0)
img_path = test_img_path + '/' + test_image_name
img = Image.open(img_path)
plt.imshow(img)
plt.title("Test Image")
plt.figure(1)
img_pre = Image.open('/kaggle/input/gtsrb-german-traffic-sign/Meta/' + str(classes_x[number_of_image_test]) + '.png')
plt.imshow(img_pre)
plt.title("Predict Image")
plt.show()

In [ ]:
#loading test images 
test = pd.read_csv(directory + '/Test.csv')

test_labels = test["ClassId"].values
test_imgs = test["Path"].values

test_data =[]

for img in tqdm(test_imgs):
    filepath = os.path.join(directory,img)
    image = Image.open(filepath)
    #resize it to 32 by 32 
    image = image.resize((32,32),Image.LANCZOS) 
    image = np.array(image).astype(np.uint8)
    #sharpen contrast
    image = exposure.equalize_adapthist(image, clip_limit=0.1) 
    #rescale back by 255 
    image = (image * 255).astype(np.uint8)
    #append the data to images_numpy 
    test_data.append(image)

In [ ]:
#convert to numpy array and check the shape of the test data
test_data = np.array(test_data)
print(test_labels.shape)
print(test_data.shape)

In [ ]:
#Produce classification report for the Test Dataset
test_prob = model.predict(test_data)
test_predicted_labels = np.argmax(test_prob, axis = 1) #take argmax because the class with the highest probability would be the predicted class
test_report = classification_report(test_labels,test_predicted_labels)
print('---')
print('Classification report for Test Dataset:')
print(test_report)

In [ ]:
# testing accuracy on test dataset
from sklearn.metrics import accuracy_score

# Importing the test dataset
y_test = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

# Retreiving the images
with tf.device('/GPU:0'):
    for img in imgs:
        image = Image.open('../input/gtsrb-german-traffic-sign/'+img)
        image = image.resize([30, 30])
        data.append(np.array(image))

X_test=np.array(data)

In [ ]:
with tf.device('/GPU:0'):
    pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

In [ ]:
def predict_image(model_path, image_path):
    # Load the model
    model = tf.keras.models.load_model(model_path)
    
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (30, 30))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    
    # Make predictions
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)
    
    return predicted_label

model_path = "/kaggle/working/saved_model.h5"
image_path = "/kaggle/input/gtsrb-german-traffic-sign/Test/00080.png"

predicted_label = predict_image(model_path, image_path)
print("Predicted Label:", classes[predicted_label])

In [ ]:
#loading test data and predictions
test = pd.read_csv(data_dir + '/Test.csv')

labels = test["ClassId"].values
imgs = test["Path"].values

data =[]

for img in imgs:
    try:
        image = cv2.imread(data_dir + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)
X_test = np.array(data)
# X_test = X_test/255


predict_x=model.predict(X_test) 
pred=np.argmax(predict_x,axis=1)



#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(labels, pred)*100)

In [ ]:
#confusion metrix
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(labels, pred)
cf

In [ ]:
#visualizing confusion metrix
import seaborn as sns
df_cm = pd.DataFrame(cf, index = classes,  columns = classes)
plt.figure(figsize = (20,20))
sns.heatmap(df_cm, annot=True)

In [ ]:
#classification report
from sklearn.metrics import classification_report

print(classification_report(labels, pred))

In [ ]:
#prediction on test data
plt.figure(figsize = (30, 30))

start_index = 0
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    prediction = pred[start_index + i]
    actual = labels[start_index + i]
    col = 'black'
    if prediction != actual:
        col = 'r'
    plt.xlabel('Actual={} || Pred={}'.format(classes[actual], classes[prediction]), color = col)
    plt.imshow(X_test[start_index + i])
plt.show()

In [ ]:
#prediction 
from PIL import Image

def predict_Image(img):
    data=[]
    image=Image.open(img).resize((30,30))
    data.append(np.array(image))
    X_test = np.array(data)
    
    predict_x=model.predict(X_test) 
    Y_pred=np.argmax(predict_x,axis=1)
    return image,Y_pred

In [ ]:
plot,prediction = predict_Image('/kaggle/input/gtsrb-german-traffic-sign/Test/00115.png')
print("\nPredicted: " ,classes[prediction.item()]) #item() converts numpy int64 to native int
plt.imshow(plot)
plt.show()